In [121]:
import pandas as pd
import re
from nltk.util import bigrams
from nltk.lm.preprocessing import pad_both_ends, padded_everygram_pipeline
from sklearn.model_selection import train_test_split
from nltk.tokenize import WhitespaceTokenizer
from nltk.lm import MLE, NgramCounter, Laplace


In [2]:
df_pt = pd.read_csv(r"D:\Databases\Files\stackoverflow_portugues.csv", sep=",")
df_en = pd.read_csv(r"D:\Databases\Files\stackoverflow_ingles.csv", sep=",")


In [83]:
display(df_pt.head())


,Id,Título,Questão,Tags,Pontuação,Visualizações,sem_code_tag,questoes_tratadas,idioma
0,2402,Como fazer hash de senhas de forma segura?,"<p>Se eu fizer o <em><a href=""http://pt.wikipe...",<hash><segurança><senhas><criptografia>,350,22367,Se eu fizer o hash de senhas antes de armazená...,se eu fizer o hash de senhas antes de armazená...,pt
1,6441,Qual é a diferença entre INNER JOIN e OUTER JOIN?,<p>Qual é a diferença entre <code>INNER JOIN</...,<sql><join>,276,176953,Qual é a diferença entre INNER JOIN e OUTER JO...,qual é a diferença entre inner join e outer jo...,pt
2,579,Por que não devemos usar funções do tipo mysql_*?,<p>Uma dúvida muito comum é por que devemos pa...,<php><mysql>,226,9761,Uma dúvida muito comum é por que devemos parar...,uma dúvida muito comum é por que devemos parar...,pt
3,2539,As mensagens de erro devem se desculpar?,<p>É comum encontrar uma mensagem de erro que ...,<aplicação-web><gui><console><ux>,214,5075,É comum encontrar uma mensagem de erro que diz...,é comum encontrar uma mensagem de erro que diz...,pt
4,17501,"Qual é a diferença de API, biblioteca e Framew...",<p>Me parecem termos muito próximos e eventual...,<api><framework><terminologia><biblioteca>,193,54191,Me parecem termos muito próximos e eventualmen...,me parecem termos muito próximos e eventualmen...,pt


In [17]:
questao_pt = df_pt["Questão"]
questao_en = df_en["Questão"]


In [85]:
# re.findall(r'<.*?>', questao_pt) achar todas ocorrencias
# re.sub(r'<.*?>', "TESTE-------", questao_pt)
regex_codigo = re.compile(r'<code>(.|(\n))*?</code>')
regex_pontuacao = re.compile(r'[^\w\s]')
regex_html = re.compile(r'<.*?>')
regex_digitos = re.compile(r'\d+')
regex_espaco = re.compile(r' +')
regex_quebra_linha = re.compile(r'(\n)')


In [9]:
def remover(textos, regex):
    if type(textos) == str:
        return regex.sub('', textos)
    else:
        return [regex.sub('', texto) for texto in textos]


In [33]:
questao_sem_tag_pt = remover(questao_pt, regex_html)
questao_sem_tag_en = remover(questao_en, regex_html)


In [21]:
def substituir_codigo(textos, regex):
    if type(textos) == str:
        return regex.sub('CODE', textos)
    else:
        return [regex.sub('CODE', texto) for texto in textos]


In [34]:
questao_sem_tag_codigo_pt = substituir_codigo(questao_sem_tag_pt, regex_codigo)
questao_sem_tag_codigo_en = substituir_codigo(questao_sem_tag_en, regex_codigo)


In [35]:
df_pt["sem_code_tag"] = questao_sem_tag_codigo_pt
df_en["sem_code_tag"] = questao_sem_tag_codigo_en


In [36]:
print(remover(questao_sem_tag_codigo_pt[0], regex_pontuacao))


Se eu fizer o hash de senhas antes de armazenálas em meu banco de dados é suficiente para evitar que elas sejam recuperadas por alguém

Estou falando apenas da recuperação diretamente do banco de dados e não qualquer outro tipo de ataque como força bruta na página de login da aplicação keylogger no cliente e criptoanálise rubberhose Qualquer forma de hash não vai impedir esses ataques

Tenho preocupação em dificultar ou até impossibilitar a obtenção das senhas originais caso o banco de dados seja comprometido Como dar maior garantia de segurança neste aspecto

Quais preocupações adicionais evitariam o acesso às senhas Existem formas melhores de fazer esse hash



In [37]:
def to_lower(textos):
    if type(textos) == str:
        return textos.lower()
    else:
        return [texto.lower() for texto in textos]


In [38]:
to_lower(questao_sem_tag_codigo_pt[0])


'se eu fizer o hash de senhas antes de armazená-las em meu banco de dados é suficiente para evitar que elas sejam recuperadas por alguém?\n\nestou falando apenas da recuperação diretamente do banco de dados e não qualquer outro tipo de ataque, como força bruta na página de login da aplicação, keylogger no cliente e criptoanálise rubberhose. qualquer forma de hash não vai impedir esses ataques.\n\ntenho preocupação em dificultar ou até impossibilitar a obtenção das senhas originais caso o banco de dados seja comprometido. como dar maior garantia de segurança neste aspecto?\n\nquais preocupações adicionais evitariam o acesso às senhas? existem formas melhores de fazer esse hash?\n'

In [47]:
def substituir_espaco(textos, regex):
    if type(textos) == str:
        return regex.sub(' ', textos)
    else:
        return [regex.sub(' ', texto) for texto in textos]


In [53]:
questoes_port_sem_pontuacao = remover(df_pt["sem_code_tag"], regex_pontuacao)
questoes_port_sem_uppercase = to_lower(questoes_port_sem_pontuacao)
questoes_port_sem_digitos = remover(questoes_port_sem_uppercase, regex_digitos)
questoes_port_sem_quebra_linha = substituir_espaco(
    questoes_port_sem_digitos, regex_quebra_linha)
questoes_port_sem_espaco = substituir_espaco(
    questoes_port_sem_quebra_linha, regex_espaco)


In [58]:
questoes_en_sem_pontuacao = remover(df_en["sem_code_tag"], regex_pontuacao)
questoes_en_sem_uppercase = to_lower(questoes_en_sem_pontuacao)
questoes_en_sem_digitos = remover(questoes_en_sem_uppercase, regex_digitos)
questoes_en_sem_quebra_linha = substituir_espaco(
    questoes_en_sem_digitos, regex_quebra_linha)
questoes_port_sem_espaco = substituir_espaco(
    questoes_en_sem_quebra_linha, regex_espaco)


In [59]:
df_en["questoes_tratadas"] = questoes_port_sem_espaco


In [55]:
df_pt["questoes_tratadas"] = questoes_port_sem_espaco


In [77]:
display(df_pt.head(1))
display(df_en.head(1))

,Id,Título,Questão,Tags,Pontuação,Visualizações,sem_code_tag,questoes_tratadas,idioma
0,2402,Como fazer hash de senhas de forma segura?,"<p>Se eu fizer o <em><a href=""http://pt.wikipe...",<hash><segurança><senhas><criptografia>,350,22367,Se eu fizer o hash de senhas antes de armazená...,se eu fizer o hash de senhas antes de armazená...,pt


,Id,Título,Questão,Tags,Pontuação,Visualizações,sem_code_tag,questoes_tratadas,idioma
0,11227809,Why is it faster to process a sorted array tha...,<p>Here is a piece of C++ code that seems very...,<java><c++><performance><optimization><branch-...,23057,1358574,Here is a piece of C++ code that seems very pe...,here is a piece of c code that seems very pecu...,en


In [67]:
print(list(bigrams(pad_both_ends("hoje", n=2))))


[('<s>', 'h'), ('h', 'o'), ('o', 'j'), ('j', 'e'), ('e', '</s>')]


In [68]:
df_pt['idioma'] = 'pt'
df_en['idioma'] = 'en'


In [75]:
display(df_en.head(1))
display(df_pt.head(1))

,Id,Título,Questão,Tags,Pontuação,Visualizações,sem_code_tag,questoes_tratadas,idioma
0,11227809,Why is it faster to process a sorted array tha...,<p>Here is a piece of C++ code that seems very...,<java><c++><performance><optimization><branch-...,23057,1358574,Here is a piece of C++ code that seems very pe...,here is a piece of c code that seems very pecu...,en


,Id,Título,Questão,Tags,Pontuação,Visualizações,sem_code_tag,questoes_tratadas,idioma
0,2402,Como fazer hash de senhas de forma segura?,"<p>Se eu fizer o <em><a href=""http://pt.wikipe...",<hash><segurança><senhas><criptografia>,350,22367,Se eu fizer o hash de senhas antes de armazená...,se eu fizer o hash de senhas antes de armazená...,pt


In [82]:
pt_treino, pt_teste = train_test_split(
    df_pt['questoes_tratadas'], test_size=0.2, random_state=123)

en_treino, en_teste = train_test_split(
    df_en['questoes_tratadas'], test_size=0.2, random_state=123)


In [89]:
pt_todas_questoes = ' '.join(pt_treino)

In [90]:
pt_todas_palavras = WhitespaceTokenizer().tokenize(pt_todas_questoes)

In [92]:
pt_treino_bigram, pt_vocab = padded_everygram_pipeline(2, pt_todas_palavras)

In [95]:
next(next(pt_treino_bigram))

('<s>',)

In [97]:
pt_modelo = MLE(2)


In [98]:
pt_modelo.fit(pt_treino_bigram, pt_vocab)

In [99]:
pt_modelo.generate(num_words=6)

['p', 'r', 'a', 'd', 'a', 'g']

In [101]:
pt_modelo.counts[['m']].items()

dict_items([('</s>', 2838), ('a', 2195), ('b', 167), ('p', 818), ('o', 913), ('i', 336), ('e', 1733), ('g', 37), ('s', 22), ('u', 233), ('v', 16), ('í', 13), ('ó', 44), ('â', 11), ('l', 52), ('c', 2), ('é', 81), ('j', 4), ('m', 43), ('ú', 5), ('ã', 4), ('á', 32), ('y', 35), ('w', 2), ('_', 3), ('x', 2), ('n', 5), ('d', 6), ('k', 1)])

In [109]:
texto = "good morning"
words = WhitespaceTokenizer().tokenize(texto)
word_fakes = [list(pad_both_ends(word, n=2)) for word in words]
word_bigrams = [list(bigrams(word)) for word in word_fakes]
print(word_bigrams)

[[('<s>', 'g'), ('g', 'o'), ('o', 'o'), ('o', 'd'), ('d', '</s>')], [('<s>', 'm'), ('m', 'o'), ('o', 'r'), ('r', 'n'), ('n', 'i'), ('i', 'n'), ('n', 'g'), ('g', '</s>')]]


In [103]:
pt_modelo.perplexity(word_bigrams[0])

14.807373528190565

In [104]:
def treinar_modelo_mle(lista_textos):
    todas_questoes = ' '.join(list(lista_textos))
    todas_palavras = WhitespaceTokenizer().tokenize(todas_questoes)
    bigrams, vocab = padded_everygram_pipeline(2, todas_palavras)

    modelo = MLE(2)
    modelo.fit(bigrams, vocab)
    return modelo


In [122]:
def treinar_modelo_laplace(lista_textos):
    todas_questoes = ' '.join(list(lista_textos))
    todas_palavras = WhitespaceTokenizer().tokenize(todas_questoes)
    bigrams, vocab = padded_everygram_pipeline(2, todas_palavras)

    modelo = Laplace(2)
    modelo.fit(bigrams, vocab)
    return modelo


In [112]:
pt_modelo = treinar_modelo_mle(pt_treino)

In [113]:
pt_modelo.perplexity(word_bigrams[0])

46.1327616894434

In [107]:
en_modelo = treinar_modelo_mle(en_treino)

In [110]:
en_modelo.perplexity(word_bigrams[0])

21.18222621860296

In [117]:
def calc_perplexidade(modelo, texto):
    perplexidade = 0
    
    words = WhitespaceTokenizer().tokenize(texto)
    word_fakes = [list(pad_both_ends(word, n=2)) for word in words]
    word_bigrams = [list(bigrams(word)) for word in word_fakes]

    for word in word_bigrams:
        perplexidade += modelo.perplexity(word)

    return perplexidade


In [119]:
print(calc_perplexidade(pt_modelo, "Good Morning"))

inf


In [127]:
pt_modelo = treinar_modelo_laplace(pt_treino)
en_modelo = treinar_modelo_laplace(en_treino)

In [129]:
print(f'pt: {calc_perplexidade(pt_modelo, pt_teste.iloc[0])}')
print(f'en: {calc_perplexidade(en_modelo, pt_teste.iloc[0])}')

pt: 2049.157443752198
en: 5875.535755191529


In [131]:
def atribuir_idioma(lt):
    idiomas = []
    for t in lt:
        pt = calc_perplexidade(pt_modelo, t)
        en = calc_perplexidade(en_modelo, t)
        if en >= pt:
            idiomas.append('pt')
        else:
            idiomas.append('en')
    return idiomas


In [132]:
pt_resultados = atribuir_idioma(pt_teste)

In [134]:
pt_resultados.count('pt')

98

In [135]:
pt_resultados.count('en')

2

In [136]:
en_resultados = atribuir_idioma(en_teste)

In [137]:
en_resultados.count('en')

100

In [139]:
en_resultados.count('pt') 

0